In [149]:
from IPython.display import display, HTML
import pandas as pd
import statsmodels.formula.api as sm
import statsmodels.regression.linear_model as lm
import matplotlib.pyplot as plt

# import numpy as np
# import plotly.express as px
# from scipy import stats
# import seaborn as sns

# Final Project: Question 2

- Which factors affect the monetary rewards of appealing? 
- What influences how much money one might gain from a successful appeal?

Utilize multivariate OLS. Our goal is to understand whether there is bias or unfairness in the tax system.

In [150]:
df = pd.read_csv('project2data.csv')

print('Dataset Description:')
print(' Original Format: comma-separated values file')
print(' Observations: Random sample of property tax appeals within Cook County')
print(' Number of observations (rows):', df.shape[0])
print(' Number of features (columns):', df.shape[1])
print(' Feature names:  ',
      ', '.join([(element + "\n\t\t") if (index % 4 ==0) else element \
                for index, element in enumerate(sorted(df.columns), start=1)]))
print(" Dependent variable: appeal")
print("   *appeal = dollar amount deducted from assessed value after property owner appealed. If NA there was no appeal.")
print(" Source: Cook County Assessor's Office")
print(' Contact: datascience@cookcountyassessor.com')

Dataset Description:
 Original Format: comma-separated values file
 Observations: Random sample of property tax appeals within Cook County
 Number of observations (rows): 19036
 Number of features (columns): 26
 Feature names:   appeal, asian, av1, avg_school_score
		, beds, black, college, condo
		, elem_score, high_school_score, hispanic, homeowner
		, medhinc, pcity, pdir, pin14
		, poverty, pstreet, squarefoot, taxes
		, tcity, tri, value, walkfac
		, walkscore, white
 Dependent variable: appeal
   *appeal = dollar amount deducted from assessed value after property owner appealed. If NA there was no appeal.
 Source: Cook County Assessor's Office
 Contact: datascience@cookcountyassessor.com


# Data Quality - Review and Cleanup

Take a high-level review of the data and remap when necessary.

In [151]:
# Review continuous features
df.describe()

,pin14,av1,value,taxes,homeowner,white,black,hispanic,asian,medhinc,poverty,college,squarefoot,beds,walkscore,elem_score,high_school_score,avg_school_score,appeal
count,1.903600e+04,1.899100e+04,1.903600e+04,19036.000000,19036.000000,18914.000000,18914.000000,18914.000000,18914.000000,18914.000000,18914.000000,18914.000000,13915.000000,13915.000000,19036.000000,19036.000000,19036.000000,19036.000000,9.018000e+03
mean,1.806325e+13,3.036896e+04,3.564597e+05,4972.586580,0.426087,0.625657,0.205279,0.186143,0.139870,67020.153960,0.135024,0.318044,1908.737478,3.648653,69.779838,5.453719,5.394883,5.644411,5.394141e+03
std,5.788279e+12,2.961622e+04,3.641909e+05,4233.282368,0.494520,0.274867,0.296543,0.214204,0.118632,26266.301109,0.099465,0.212173,1167.077974,1.701771,23.504392,2.406259,2.865633,2.223499,2.029286e+04
min,9.253050e+12,-1.045572e+06,5.000000e+03,0.000000,0.000000,0.001088,0.000789,0.001889,0.000000,9550.000000,0.003604,0.004178,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000e+00
25%,1.418130e+13,1.534750e+04,1.750000e+05,2479.315050,0.000000,0.499139,0.020925,0.053135,0.053844,47972.000000,0.065292,0.134969,1151.000000,3.000000,55.000000,3.000000,3.000000,4.000000,0.000000e+00
50%,1.704110e+13,2.367500e+04,2.660000e+05,3940.760000,0.000000,0.703167,0.051095,0.091340,0.099713,63529.000000,0.112022,0.265332,1567.000000,3.000000,75.000000,6.000000,5.000000,6.000000,1.177500e+03
75%,2.016100e+13,3.680350e+04,4.130000e+05,6088.760125,1.000000,0.851731,0.261715,0.217768,0.194202,84167.000000,0.170868,0.505770,2297.000000,4.000000,89.000000,7.000000,8.000000,7.000000,4.375000e+03
max,3.331210e+13,7.639090e+05,1.500000e+07,133258.690000,1.000000,0.970822,0.993390,0.979010,0.896609,192143.000000,0.776376,0.766168,27270.000000,24.000000,100.000000,10.000000,10.000000,10.000000,1.164486e+06


In [152]:
# Drop duplicate rows
print("Total df row count: ",len(df))
print("Row count after dropping duplicate rows: ", len(df.drop_duplicates()))
df = df.drop_duplicates()

Total df row count:  19036
Row count after dropping duplicate rows:  18904


In [153]:
# Data Quality Review for each feature individually and remap when necessary
for feature in sorted(list(df.columns)):
    # Feature Summary section
    print(feature,':  data type:',str(df.dtypes[feature]))
    print('       unique values:',len(df[feature].unique()))
    if len(df[feature].unique()) < 20 and not isinstance(df[feature], str):
        print("    Original unique values for review:", str(df[feature].unique()))
    
    if feature == 'walkfac':
        Walkfac_dict = {'Car-Dependent': '1',
                        'Somewhat Walkable': '2',
                        'Very Walkable': '3',
                        "Walker's Paradise": '4'
                            }
        df[feature] = df[feature].map(Walkfac_dict)
        print("    New unique values for review:", str(sorted(df[feature].unique())))
    if feature == 'condo':
        Walkfac_dict = {'Non-condo': 0,
                        'Condominium': 1
                            }
        df[feature] = df[feature].map(Walkfac_dict)
        print("    New unique values for review:", str(sorted(df[feature].unique())))
    if feature == 'tri':
        # Convert data type to category
        df[feature] = df[feature].astype('category')
        print("New data type:",str(df.dtypes[feature]) )    
    if feature == 'pin14':
        # Convert data type to category
        df[feature] = df[feature].astype('str')
        print("New data type:",str(df.dtypes[feature]))

appeal :  data type: float64
       unique values: 4302
asian :  data type: float64
       unique values: 1108
av1 :  data type: float64
       unique values: 15796
avg_school_score :  data type: int64
       unique values: 10
    Original unique values for review: [ 4  9  2  6  8  3  7  5  1 10]
beds :  data type: float64
       unique values: 21
black :  data type: float64
       unique values: 1110
college :  data type: float64
       unique values: 1109
condo :  data type: object
       unique values: 2
    Original unique values for review: ['Non-condo' 'Condominium']
    New unique values for review: [0, 1]
elem_score :  data type: int64
       unique values: 10
    Original unique values for review: [ 1 10  5  8  2  7  4  9  6  3]
high_school_score :  data type: int64
       unique values: 10
    Original unique values for review: [ 6  8  2 10  1  5  4  9  7  3]
hispanic :  data type: float64
       unique values: 1110
homeowner :  data type: int64
       unique values: 2
    Or

In [154]:
# Break pin14 feature into subcomponents and drop
# Per docs:
# All PINs are 14 digits: 2 digits for area + 2 digits for sub area + 2 digits for block + 2 digits for parcel + 4 digits for the multicode
# NOTE: Docs do not account for meaning of full 14 digits (above adds up to 12; unclear of any meaning for last 2 digits)
df['area']      = df.pin14.str[0:2]
df['subarea']   = df.pin14.str[2:4]
df['block']     = df.pin14.str[4:6]
df['parcel']    = df.pin14.str[6:8]
df['multicode'] = df.pin14.str[8:12]
df = df.drop(columns='pin14')

# Review feature unique values
for feature in ['area','subarea','block','parcel','multicode']:
    # Feature Summary section
    print(feature, '  unique values:',len(df[feature].unique()))
    if len(df[feature].unique()) < 20 and not isinstance(df[feature], str):
        print("    Original unique values for review:", str(df[feature].unique()))

# Drop less useful features based on low cardinality
df = df.drop(columns='parcel')
df = df.drop(columns='multicode')

area   unique values: 26
subarea   unique values: 42
block   unique values: 41
parcel   unique values: 1
    Original unique values for review: ['00']
multicode   unique values: 1
    Original unique values for review: ['0000']


## Data Quality - Missingness

Assess and handle missing data

In [155]:
# Review missingness
print("Feature_________________Missing Value Count")
print(df.isna().sum())
print("\ndf row count:",len(df))

Feature_________________Missing Value Count
tri                     0
pdir                 5439
pstreet                 1
pcity                   1
tcity                   0
condo                   0
av1                    45
value                   0
taxes                   0
homeowner               0
white                 122
black                 122
hispanic              122
asian                 122
medhinc               122
poverty               122
college               122
squarefoot           5118
beds                 5118
walkscore               0
walkfac                 0
elem_score              0
high_school_score       0
avg_school_score        0
appeal               9982
area                    0
subarea                 0
block                   0
dtype: int64

df row count: 18904


In [156]:
# We only need those who appealed for this question (>=0); drop those who did not appeal (NaN)
df = df[df.appeal.notnull()]
print("Row count after dropping null appeal observations:",len(df))

Row count after dropping null appeal observations: 8922


In [157]:
# Drop features based on missingness (>10%) (not candidate for imputation)
## squarefoot and beds are each missing 2567 observations; missingness has 1-1 correlation
df = df.drop(columns='pdir')       # Missing 2353/8922 
df = df.drop(columns='squarefoot') # Missing 2567/8922 
df = df.drop(columns='beds')       # Missing 2567/8922 

In [158]:
# Manual review finds missingness is 1-1 for white, black, hispanic, asian, medhinc, poverty, and college features.
## Missing 91/8922 observations (~1%); opted to drop these.
df = df[df.college.notnull()]

In [159]:
# Review missingness of data; final row count 8831
print("Feature_________________Missing Value Count")
print(df.isna().sum())
print("\ndf row count:",len(df))

Feature_________________Missing Value Count
tri                  0
pstreet              0
pcity                0
tcity                0
condo                0
av1                  0
value                0
taxes                0
homeowner            0
white                0
black                0
hispanic             0
asian                0
medhinc              0
poverty              0
college              0
walkscore            0
walkfac              0
elem_score           0
high_school_score    0
avg_school_score     0
appeal               0
area                 0
subarea              0
block                0
dtype: int64

df row count: 8831
